In [2]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 48.3 MB/s 


In [3]:
pip install PyKomoran

     |████████████████████████████████| 6.4 MB 4.3 MB/s 
     |████████████████████████████████| 198 kB 42.9 MB/s 


In [4]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from PyKomoran import *

In [5]:
komoran = Komoran("STABLE")

In [14]:
# 리뷰 데이터 가져오기
tagged_review = pd.read_csv('positive_all.csv')
df_reviews = tagged_review["review_positive"]


In [15]:
df_cleansing = pd.DataFrame(
        columns=["sentence", "cleansed_target", "cleansed_stopwords"]
    )

In [44]:
df_reviews

0                                          직원분들이 다들 친절하셨어요
1        아침뷔페 및 바다가 보이는 수영장, 사우나 시설 모두 탑티어 입니다.  비용이 부담...
2                                           위치 서비스 전부 만족해요
3        조식 만족스러웠습니다. 객실내 조명 불량으로 불이 꺼지지않는 상황에서 새벽에 문의를...
4                                        깔끔하고 서비스 다 너무좋았어요
                               ...                        
43141    서귀포 여행시 위치도 너무 좋고 주변 상권도 잘되있고 식당도 많아서 추천하는 곳입니...
43142    주변에 아침 먹을 식당이 마땅치 않아 어쩔 수 없이 조식 부페를 사먹음. 17000...
43143    숙소 주변에 음식점, 편의점, 술집, 카페, 마트, 맥도날드 다 있습니다. 덕분에 ...
43144    가격대비 가성비도 좋고 호텔도 깨끗하고 모든게 좋았습니다. 바빠서 사우나를 이용못한...
43145                                      위치도 좋고 뷰도 좋았습니다
Name: review_positive, Length: 43146, dtype: object

In [16]:
def remove_stopwords(document):
    target_tags = ["NNP", "NNG", "VV", "VA", "EC"]
    cleansed_list = komoran.get_morphes_by_tags(document, tag_list=target_tags)

    stop_words = "이	있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 소리 놓"

    stop_words = set(stop_words.split(" "))

    result = [word for word in cleansed_list if not word in stop_words]

    return cleansed_list, result

In [17]:
# review_list를 가져와서 명사만 추출한 후 tag(index)와 함께 tagged_corpus_list에 저장
tagged_corpus_list = []
for i, document in enumerate(df_reviews):
    cleansed_target, cleansed_stopwords = remove_stopwords(document)
    try:
        tagged_line = TaggedDocument(tags=[i], words=cleansed_stopwords)
        #tagged_line = TaggedDocument(tags=[i], words=okt.morphs(document))
        tagged_corpus_list.append(tagged_line)
    except Exception as e:
        print('Error in tagged_corpus')
        continue
    print(f'---{round((i / len(df_reviews)) * 100 ,5)}%---')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
---88.411%---
---88.414%---
---88.416%---
---88.41799999999999%---
---88.421%---
---88.423%---
---88.425%---
---88.428%---
---88.42999999999999%---
---88.432%---
---88.435%---
---88.437%---
---88.43900000000001%---
---88.442%---
---88.444%---
---88.446%---
---88.449%---
---88.45100000000001%---
---88.453%---
---88.455%---
---88.458%---
---88.46000000000001%---
---88.46199999999999%---
---88.465%---
---88.467%---
---88.469%---
---88.472%---
---88.474%---
---88.476%---
---88.479%---
---88.481%---
---88.483%---
---88.486%---
---88.488%---
---88.49000000000001%---
---88.493%---
---88.495%---
---88.497%---
---88.5%---
---88.50200000000001%---
---88.504%---
---88.506%---
---88.509%---
---88.511%---
---88.51299999999999%---
---88.51599999999999%---
---88.518%---
---88.52%---
---88.523%---
---88.52499999999999%---
---88.527%---
---88.53%---
---88.532%---
---88.534%---
---88.537%---
---88.539%---
---88.541%---
---88.544%---
---88.546%---
---88.548%---
---88.5

In [18]:
print(tagged_corpus_list[-3:])

[TaggedDocument(words=['숙소', '주변', '음식점', '편의점', '술집', '카페', '마트', '맥도날드', '덕분', '밤', '어둡', '고', '위치', '만족', '객실', '화장실', '청결'], tags=[43143]), TaggedDocument(words=['가격', '대비', '성비', '고', '호텔', '고', '게', '바쁘', '아서', '사우나', '이용', 'ㄴ게', '안타깝'], tags=[43144]), TaggedDocument(words=['위치', '고', '뷰'], tags=[43145])]


In [19]:
# 모델 생성
d2v_model = Doc2Vec(vector_size=300, window=3, workers=8, min_count = 2)

In [20]:
# 단어 빌드
d2v_model.build_vocab(tagged_corpus_list)

In [21]:
# 학습
d2v_model.train(tagged_corpus_list, total_examples=d2v_model.corpus_count, epochs=30)

In [34]:
# 테스트 (무난한 값)
user_input = '바다가 가깝고 직원 분들이 친절하셨습니다'

cleansed_test_target, cleansed_test_stopwords = remove_stopwords(user_input)
noun_user_input = cleansed_test_stopwords
print(noun_user_input)

['바다', '가깝', '고', '직원', '친절']


In [35]:
# 유저 입력값의 벡터 생성
user_input_vector = d2v_model.infer_vector(noun_user_input)

In [36]:
# 유사도가 높은 리뷰 확인
most_similar_docs = d2v_model.docvecs.most_similar([user_input_vector], topn=5)

In [37]:
# 해당 리뷰 보기
for index, similarity in most_similar_docs:
    print(f'index :{index}')
    print(f'similarity : {similarity}')
    print(f'Nouns: {tagged_corpus_list[index]}')
    print(f'review : {df_reviews[index]}')
    print(f'------------------------')

index :15556
similarity : 0.7921287417411804
Nouns: TaggedDocument(['바다', '가깝', '고', '직원', '친절'], [15556])
review : 바다에 가깝고 직원이 친절함
------------------------
index :36636
similarity : 0.7549945116043091
Nouns: TaggedDocument(['위치', '바다', '직원', '친절'], [36636])
review : 위치, 바다, 직원 친절
------------------------
index :23660
similarity : 0.7148313522338867
Nouns: TaggedDocument(['바다', '직원', '친절'], [23660])
review : 바다가 정말 잘 보인다. 직원분들 친절합니다. ^^
------------------------
index :22618
similarity : 0.7025419473648071
Nouns: TaggedDocument(['바다', '고', '위치', '고', '친절'], [22618])
review : 바다가 매우 잘보이고 위치도 좋고 친절하십니다.
------------------------
index :11605
similarity : 0.6844241619110107
Nouns: TaggedDocument(['위치', '바다', '가깝', '어서', '고', '시설', '직원', '친절'], [11605])
review : 위치가 바다와 가까워서 좋고, 시설이 매우 깨끗했습니다. 직원 분들도 친절하셨어요.
------------------------


In [42]:
# 테스트 (짧거나 많이 나올만한 문장)
user_input = '조식'

cleansed_test_target, cleansed_test_stopwords = remove_stopwords(user_input)
noun_user_input = cleansed_test_stopwords
print(noun_user_input)

['조식']


In [43]:
# 유저 입력값의 벡터 생성
user_input_vector = d2v_model.infer_vector(noun_user_input)
# 유사도가 높은 리뷰 확인
most_similar_docs = d2v_model.docvecs.most_similar([user_input_vector], topn=5)
# 해당 리뷰 보기
for index, similarity in most_similar_docs:
    print(f'index :{index}')
    print(f'similarity : {similarity}')
    print(f'Nouns: {tagged_corpus_list[index]}')
    print(f'review : {df_reviews[index]}')
    print(f'------------------------')

index :26633
similarity : 0.9125255942344666
Nouns: TaggedDocument(['조식'], [26633])
review : 조식
------------------------
index :38870
similarity : 0.8996597528457642
Nouns: TaggedDocument(['조식'], [38870])
review : 조식이 너무좋았음.
------------------------
index :653
similarity : 0.8955320119857788
Nouns: TaggedDocument(['조식'], [653])
review : 조식 좋았음
------------------------
index :38479
similarity : 0.8920850157737732
Nouns: TaggedDocument(['조식'], [38479])
review : 조식은 좋았습니다.
------------------------
index :29589
similarity : 0.8899180889129639
Nouns: TaggedDocument(['조식'], [29589])
review : 조식
------------------------
